In [1]:
import numpy as np
import pandas as pd
import mysql.connector
from mysql.connector import Error
import yaml

In [2]:
connection = mysql.connector.connect(
    host = "192.168.50.120",
    port = "3306",
    user = "james",
    passwd = "longbowL63",
    database = "bbl_cricket"
)

cursor = connection.cursor()

In [3]:
def execute_query(query):
    # print(query)
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [4]:
def read_query(query):
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

# Store List of Games

In [5]:
with open("bbl_json/README.txt", "r") as file:
    strings = file.readlines()
    data = strings[24::]
    data = [i.split(" - ") for i in data]
    data = [i[0:5] + [i[5][:-1].split(" vs ")[0]] + [i[5][:-1].split(" vs ")[1]] for i in data]
    data = pd.DataFrame(
        {i[0]:i[1::] for i in data}, 
        index=["Club", "Game", "Gender", "Id", "Team_1", "Team_2"]
        ).transpose()
    data = data[["Team_1", "Team_2", "Id"]]

data["Date"] = data.index

In [6]:
create_table_query = """
create table games (
    id INT PRIMARY KEY,
    date DATE,
    team_1 VARCHAR(20),
    team_2 VARCHAR(20)
);
"""
execute_query(create_table_query)

Error: '1050 (42S01): Table 'games' already exists'


In [7]:
insert_to_table = "INSERT INTO games VALUES "

count = data.shape[0]

for i in range(count): #data.shape[0]):
    insert_to_table += "({}, '{}', '{}', '{}'), ".format(
    data.iloc[i]["Id"], 
    data.iloc[i]["Date"], 
    data.iloc[i]["Team_1"], 
    data.iloc[i]["Team_2"])

insert_to_table = insert_to_table[:-2] + ";"
# print(insert_to_table)
execute_query(insert_to_table)

Error: '1062 (23000): Duplicate entry '1269675' for key 'PRIMARY''


# Store match data

In [8]:
def store_game_info(info):
    to_store = pd.DataFrame()
    to_store.loc[index, "ID"] = index
    to_store.loc[index, "Date"] = info["dates"][0]
    to_store.loc[index, "season"] = info["season"]
    to_store.loc[index, "ballsPerOver"] = info["balls_per_over"]
    to_store.loc[index, "overs"] = info["overs"]
    to_store.loc[index, "venue"] = info["venue"]
    to_store.loc[index, "matchNumber"] = info["event"]["match_number"]
    to_store.loc[index, "matchType"] = info["match_type"]
    to_store.loc[index, "team_1"] = info["teams"][0]
    to_store.loc[index, "team_2"] = info["teams"][1]
    to_store.loc[index, "winner"] = info["outcome"]["winner"]
    to_store.loc[index, "win_by"] = info["outcome"]["by"]["runs"]
    for i in range(len(info["player_of_match"])):
        to_store.loc[index, "playerOfMatch_{}".format(i)] = info["player_of_match"][i]
    to_store.loc[index, "tossWinner"] = info["toss"]["winner"]
    to_store.loc[index, "tossDecision"] = info["toss"]["decision"]
    for i in range(len(info["officials"]["match_referees"])):
        to_store.loc[index, "referee_{}".format(i)] = info["officials"]["match_referees"][i]
    for i in range(len(info["officials"]["umpires"])):
        to_store.loc[index, "umpire_{}".format(i)] = info["officials"]["umpires"][i]

    # def dataframe_to_sql(dataframe):
    sql_dtype = {"object":"VARCHAR(20)", "float64":"INT"}
    primary_key = "ID"
    command = "create table if not exists matchData ("
    for i in to_store.columns:
        if i == primary_key:
            command += "{} {} PRIMARY KEY, ".format(i, sql_dtype["{}".format(to_store[i].dtype)])
        else:
                command += "{} {}, ".format(i, sql_dtype["{}".format(to_store[i].dtype)]) 
    command = command[:-2] + ");"
    execute_query(command)

In [9]:
def to_sql(dataframe, tablename):
    command = "INSERT INTO {} VALUES ".format(tablename)
    for i in range(dataframe.shape[0]):
        command += "{}, ".format(tuple(dataframe.iloc[i].values))

    command = command.replace('nan', "NULL")
    command = command.replace('None', "NULL")
    return command[:-2] + ";"

def store_match_data():
    play_by_play = pd.DataFrame()
    teams = [i["team"] for i in data["innings"]]

    count = 0
    inning_num = 0
    for inning in data["innings"]:
        inning_num += 1
        for over in inning["overs"]:
            ball_count = 0
            for delivery in over['deliveries']:
                ball_count+= 1
                play_by_play.loc[count, "ball"] = ball_count
                play_by_play.loc[count, "ID"] = index
                play_by_play.loc[count, "inning"] = inning_num
                play_by_play.loc[count, "batting_team"] = inning["team"]
                play_by_play.loc[count, "bowling_team"] = teams[1] if inning["team"] == teams[0] else teams[0]
                play_by_play.loc[count, "over_num"] = over['over']
                play_by_play.loc[count, "batsman"] = delivery['batter']
                play_by_play.loc[count, "bowler"] = delivery['bowler']
                play_by_play.loc[count, "non_striker"] = delivery['non_striker']
                play_by_play.loc[count, "batsman_runs"] = delivery['runs']['batter']
                play_by_play.loc[count, "extra_runs"] = delivery['runs']['extras']
                play_by_play.loc[count, "total_runs"] = delivery['runs']['total']
                play_by_play.loc[count, "extras_type"] = list(delivery['extras'])[0] if 'extras' in delivery.keys() else None
                play_by_play.loc[count, "is_wicket"] = 1 if "wickets" in delivery.keys() else 0
                if 'wickets' in delivery.keys():
                    play_by_play.loc[count, "dismissal_kind"] = delivery['wickets'][0]['kind']
                    play_by_play.loc[count, "player_dismissed"] = delivery['wickets'][0]['player_out']
                    if 'fielders' in delivery['wickets'][0].keys():
                        play_by_play.loc[count, "fielder"] = delivery['wickets'][0]['fielders'][0]["name"]
                count += 1
    play_by_play["ID"] = play_by_play["ID"].astype(np.int64)
    play_by_play["PlayID"] =play_by_play.index
    # def dataframe_to_sql(dataframe):
    sql_dtype = {"object":"VARCHAR(20)", "float64":"INT", "int64":"INT"}

    primary_key = "PlayID"
    foreign_key = "ID"
    reference = "games(id)"
    command = "create table if not exists matchData("
    for i in play_by_play.columns:
        # print(i, play_by_play[i].dtype)
        if i == primary_key:
            command += "{} {} PRIMARY KEY, ".format(i, sql_dtype["{}".format(play_by_play[i].dtype)])
        else:
            command += "{} {}, ".format(i, sql_dtype["{}".format(play_by_play[i].dtype)])
    command += "constraint fk_id_new foreign key({}) references {}".format(foreign_key, reference)  
    command = command + ");"
    execute_query(command)

    execute_query(to_sql(play_by_play, "matchData"))
        

In [12]:
for i in range(data.shape[0]):
    index = data.iloc[0]["Id"]
    with open("bbl_json/{}.json".format(index), "r") as file:
        match_data = yaml.safe_load(file)
        file.close()
    info = data['info']
    store_game_info(info)
    

In [14]:
index = data.iloc[0]["Id"]

In [15]:
with open("bbl_json/{}.json".format(index), "r") as file:
    data = yaml.safe_load(file)
    file.close()


In [16]:
info = data['info']
to_store = pd.DataFrame()

In [17]:
to_store.loc[index, "ID"] = index
to_store.loc[index, "Date"] = info["dates"][0]
to_store.loc[index, "season"] = info["season"]
to_store.loc[index, "ballsPerOver"] = info["balls_per_over"]
to_store.loc[index, "overs"] = info["overs"]
to_store.loc[index, "venue"] = info["venue"]
to_store.loc[index, "matchNumber"] = info["event"]["match_number"]
to_store.loc[index, "matchType"] = info["match_type"]
to_store.loc[index, "team_1"] = info["teams"][0]
to_store.loc[index, "team_2"] = info["teams"][1]
to_store.loc[index, "winner"] = info["outcome"]["winner"]
to_store.loc[index, "win_by"] = info["outcome"]["by"]["runs"]
for i in range(len(info["player_of_match"])):
    to_store.loc[index, "playerOfMatch_{}".format(i)] = info["player_of_match"][i]
to_store.loc[index, "tossWinner"] = info["toss"]["winner"]
to_store.loc[index, "tossDecision"] = info["toss"]["decision"]
for i in range(len(info["officials"]["match_referees"])):
    to_store.loc[index, "referee_{}".format(i)] = info["officials"]["match_referees"][i]
for i in range(len(info["officials"]["umpires"])):
    to_store.loc[index, "umpire_{}".format(i)] = info["officials"]["umpires"][i]

In [18]:
# def dataframe_to_sql(dataframe):
sql_dtype = {"object":"VARCHAR(20)", "float64":"INT"}

primary_key = "ID"

command = "create table matchData ("
for i in to_store.columns:
    if i == primary_key:
        command += "{} {} PRIMARY KEY, ".format(i, sql_dtype["{}".format(to_store[i].dtype)])
    else:
            command += "{} {}, ".format(i, sql_dtype["{}".format(to_store[i].dtype)])
            
command = command[:-2] + ");"

execute_query(command)

Error: '1050 (42S01): Table 'matchData' already exists'


# Store play by plays

In [19]:
columns = [
    "PlayID",
    "ID",
    "inning",
    "over",
    "ball",
    "batsman",
    "non-striker",
    "bowler",
    "batsman_runs",
    "extra_runs",
    "total_runs",
    "is_wicket",
    "dismissal_kind",
    "player_dismissed",
    "fielder",
    "extras_type",
    "batting_team",
    "bowling_team"
]

play_by_play = pd.DataFrame()

In [20]:
teams = [i["team"] for i in data["innings"]]

count = 0
inning_num = 0
for inning in data["innings"]:
    inning_num += 1
    for over in inning["overs"]:
        ball_count = 0
        for delivery in over['deliveries']:
            ball_count+= 1
            play_by_play.loc[count, "ball"] = ball_count
            play_by_play.loc[count, "ID"] = index
            play_by_play.loc[count, "inning"] = inning_num
            play_by_play.loc[count, "batting_team"] = inning["team"]
            play_by_play.loc[count, "bowling_team"] = teams[1] if inning["team"] == teams[0] else teams[0]
            play_by_play.loc[count, "over_num"] = over['over']
            play_by_play.loc[count, "batsman"] = delivery['batter']
            play_by_play.loc[count, "bowler"] = delivery['bowler']
            play_by_play.loc[count, "non_striker"] = delivery['non_striker']
            play_by_play.loc[count, "batsman_runs"] = delivery['runs']['batter']
            play_by_play.loc[count, "extra_runs"] = delivery['runs']['extras']
            play_by_play.loc[count, "total_runs"] = delivery['runs']['total']
            play_by_play.loc[count, "extras_type"] = list(delivery['extras'])[0] if 'extras' in delivery.keys() else None
            play_by_play.loc[count, "is_wicket"] = 1 if "wickets" in delivery.keys() else 0
            if 'wickets' in delivery.keys():
                play_by_play.loc[count, "dismissal_kind"] = delivery['wickets'][0]['kind']
                play_by_play.loc[count, "player_dismissed"] = delivery['wickets'][0]['player_out']
                if 'fielders' in delivery['wickets'][0].keys():
                    play_by_play.loc[count, "fielder"] = delivery['wickets'][0]['fielders'][0]["name"]
            count += 1

In [88]:
play_by_play["ID"] = play_by_play["ID"].astype(np.int64)

In [89]:
play_by_play["ID"] = play_by_play["ID"].astype(np.int64)
play_by_play["PlayID"] =play_by_play.index

In [91]:
test = play_by_play["PlayID"].value_counts()
test[test > 1]

Series([], Name: PlayID, dtype: int64)

In [93]:
# def dataframe_to_sql(dataframe):
sql_dtype = {"object":"VARCHAR(20)", "float64":"INT", "int64":"INT"}

primary_key = "PlayID"
foreign_key = "ID"
reference = "games(id)"

command = "create table matchData("
for i in play_by_play.columns:
    # print(i, play_by_play[i].dtype)
    if i == primary_key:
        command += "{} {} PRIMARY KEY, ".format(i, sql_dtype["{}".format(play_by_play[i].dtype)])
    else:
        command += "{} {}, ".format(i, sql_dtype["{}".format(play_by_play[i].dtype)])

# command += "{} {}, ".format("ID", sql_dtype["{}".format(play_by_play["ID"].dtype)])

command += "constraint fk_id_new foreign key({}) references {}".format(foreign_key, reference)
            
command = command + ");"
print(command)
print("\n")
execute_query("drop table if exists matchData;")
execute_query(command)

create table matchData(ball INT, ID INT, inning INT, batting_team VARCHAR(20), bowling_team VARCHAR(20), over_num INT, batsman VARCHAR(20), bowler VARCHAR(20), non_striker VARCHAR(20), batsman_runs INT, extra_runs INT, total_runs INT, extras_type VARCHAR(20), is_wicket INT, dismissal_kind VARCHAR(20), player_dismissed VARCHAR(20), fielder VARCHAR(20), PlayID INT PRIMARY KEY, constraint fk_id_new foreign key(ID) references games(id));


Query successful
Query successful


In [113]:
def to_sql(dataframe, tablename):
    command = "INSERT INTO {} VALUES ".format(tablename)
    for i in range(dataframe.shape[0]):
        command += "{}, ".format(tuple(dataframe.iloc[i].values))

    command = command.replace('nan', "NULL")
    command = command.replace('None', "NULL")
    return command[:-2] + ";"
        

In [114]:
test = to_sql(play_by_play, "matchData")

In [116]:
test.split('(');

In [117]:
execute_query(test)

Query successful
